In [1]:
import imp
import sys
sys.path.append('../lib')
from outlier import *
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from itertools import groupby
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest

import os

In [2]:
store_cible=pd.HDFStore('../PCA/cible_crossB2B_1.h5')
uselessFeatures=list(pd.read_table('./uselessFeatures.csv', sep=',')['name'])
dg=store_cible.select('df201606_sample200000')
cible=dg.fin
dg=dg.drop(uselessFeatures,axis=1)
dg_drop=dg.drop('fin',axis=1)

In [3]:
dg_drop_minmax=pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(dg_drop),columns=dg_drop.columns)

In [4]:
normal_dataset=np.array(dg_drop_minmax)[np.where(np.array(dg['fin'])==1)]

# Kmeans

In [ ]:
result_file='result_grid_kmeans.csv'
dir_path='./'

### Sans réduction de dimension

In [ ]:
text_file = open(os.path.join(dir_path, result_file), 'w')
normal_dataset=np.array(dg_drop_minmax)[np.where(np.array(dg['fin'])==1)]
for k in range(3,12):
    
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10,random_state=5)
    pipeline=Pipeline([ ('removeOutliers',OutliersKmeans(parallel=True,kmeans=kmeans,showbar=True))])


    tuned_parameters = [{'penalty': ['l1','l2'],
            'C':list(np.logspace(-4, 4, 10))}]


    clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=10)
    fittedpipeline=pipeline.fit(normal_dataset)
    data=fittedpipeline.transform(np.array(dg_drop_minmax))
    cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]
    clf.fit(data,cible_dropt)


    print('--------------------------', file=text_file)
    print("K :"+str(k), file=text_file)

    print('', file=text_file)
    print("Best parameters set found on development set:", file=text_file)
    print('', file=text_file)
    print(clf.best_params_, file=text_file)
    print('', file=text_file)

    print("Grid scores on development set:", file=text_file)
    print('', file=text_file)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params), file=text_file)
    print('', file=text_file)

text_file.close()

### Avec réduction de dimension 

In [ ]:
result_file='result_grid_kmeans_reduction.csv'
dir_path='./'

In [ ]:
text_file = open(os.path.join(dir_path, result_file), 'w')
normal_dataset=np.array(dg_drop_minmax)[np.where(np.array(dg['fin'])==1)]
for k in range(3,15):
    for ncompo in range(2,10):
    
        kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10,random_state=5)
        pipeline=Pipeline([('dim_red',PCA(n_components=ncompo,svd_solver='arpack')),
                           ('removeOutliers',OutliersKmeans(parallel=True,kmeans=kmeans,showbar=False))])


        tuned_parameters = [{'penalty': ['l1','l2'],
                'C':list(np.logspace(-4, 4, 10))}]


        clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=1)
        fittedpipeline=pipeline.fit(normal_dataset)
        data=fittedpipeline.transform(np.array(dg_drop_minmax)) 
        cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]
        clf.fit(data,cible_dropt)


        print('--------------------------', file=text_file)
        print("K :"+str(k), file=text_file)
        print("Number component :"+str(ncompo), file=text_file)

        print('', file=text_file)
        print("Best parameters set found on development set:", file=text_file)
        print('', file=text_file)
        print(clf.best_params_, file=text_file)
        print('', file=text_file)

        print("Grid scores on development set:", file=text_file)
        print('', file=text_file)
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params), file=text_file)
        print('', file=text_file)

text_file.close()

# Isolation Forest

In [ ]:
class OutliersIsolationForest(TransformerMixin):

    def __init__(self,forest,showbar=False):
        self.forest=forest
        self.showbar=showbar

    def get_params(self):
        return {"forest": self.forest}

    def set_params(self, **parameters):
        for key,value in parameters.items() :
            setattr(self,key,parameters[key])
        return self


    def fit(self,X,y=None):

        self.forest.fit(X)

        return self

    def transform(self,X):

        a=self.forest.predict(X)
        self.iextreme_values=a==-1
        
        
        return X[~self.iextreme_values]


### Sans réduction de dimension

In [7]:
result_file='result_grid_iso.csv'
dir_path='./'

In [8]:
text_file = open(os.path.join(dir_path, result_file), 'w')

for contamination in [0.1,0.2,0.3,0.4,0.5]:
    for esti in [100,200,500]:
    
        iso = IsolationForest(contamination=contamination,n_estimators=esti)
        pipeline=Pipeline([ ('removeOutliers',OutliersIsolationForest(iso))])


        tuned_parameters = [{'penalty': ['l1','l2'],
                'C':list(np.logspace(-4, 4, 10))}]


        clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=10)
        fittedpipeline=pipeline.fit(np.array(dg_drop_minmax))
        data=fittedpipeline.transform(np.array(dg_drop_minmax))
        cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]
        clf.fit(data,cible_dropt)


        print('--------------------------', file=text_file)
        print("Estimators :"+str(esti), file=text_file)
        print("Contamination :"+str(contamination), file=text_file)

        print('', file=text_file)
        print("Best parameters set found on development set:", file=text_file)
        print('', file=text_file)
        print(clf.best_params_, file=text_file)
        print('', file=text_file)

        print("Grid scores on development set:", file=text_file)
        print('', file=text_file)
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params), file=text_file)
        print('', file=text_file)

text_file.close()

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.8s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.9s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.0s remaining:    0.0s


[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.8s
[CV] penalty=l1, C=0.0001 ............................................


...

[CV] ............. penalty=l2, C=0.0001, score=0.451887, total=   0.6s
[CV] penalty=l2, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    5.2s remaining:    0.0s


[CV] ............. penalty=l2, C=0.0001, score=0.503782, total=   0.5s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   0.4s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   0.4s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   0.4s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   0.4s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   0.4s
[CV] penalty=l2, C=0.000774263682681 .................................
[CV] .. penalty=l2, C=0.000774263682681, score=0.377120, total=   0.6s
[CV] penalty=l2, C=0.000774263682681 .................................
[CV] .

[CV] ...... penalty=l2, C=21.5443469003, score=0.690482, total=   3.1s
[CV] penalty=l2, C=21.5443469003 .....................................
[CV] ...... penalty=l2, C=21.5443469003, score=0.639654, total=   3.6s
[CV] penalty=l2, C=21.5443469003 .....................................
[CV] ...... penalty=l2, C=21.5443469003, score=0.712526, total=   3.5s
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.731920, total= 1.5min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.686811, total= 1.2min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.690116, total= 1.4min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.632712, total= 1.3min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 40.3min finished


### Avec réduction de dimension

In [11]:
result_file='result_grid_iso_reduction.csv'
dir_path='./'

In [12]:
text_file = open(os.path.join(dir_path, result_file), 'w')

for contamination in [0.1,0.2,0.3,0.4,0.5]:
    for esti in [100,200,500]:
        for compo in range(2,10):
    
            iso = IsolationForest(contamination=contamination,n_estimators=esti)
            pipeline=Pipeline([('dim_red',PCA(n_components=compo,svd_solver='arpack')),
                           ('removeOutliers',OutliersIsolationForest(iso))])



            tuned_parameters = [{'penalty': ['l1','l2'],
                    'C':list(np.logspace(-4, 4, 10))}]


            clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=10)
            fittedpipeline=pipeline.fit(np.array(dg_drop_minmax))
            data=fittedpipeline.transform(np.array(dg_drop_minmax))
            cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]
            clf.fit(data,cible_dropt)


            print('--------------------------', file=text_file)
            print("Components :"+str(compo), file=text_file)
            print("Estimators :"+str(esti), file=text_file)
            print("Contamination :"+str(contamination), file=text_file)

            print('', file=text_file)
            print("Best parameters set found on development set:", file=text_file)
            print('', file=text_file)
            print(clf.best_params_, file=text_file)
            print('', file=text_file)

            print("Grid scores on development set:", file=text_file)
            print('', file=text_file)
            means = clf.cv_results_['mean_test_score']
            stds = clf.cv_results_['std_test_score']
            for mean, std, params in zip(means, stds, clf.cv_results_['params']):
                print("%0.3f (+/-%0.03f) for %r"
                    % (mean, std * 2, params), file=text_file)
            print('', file=text_file)

text_file.close()

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.0s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.0s
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.0s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.6s remaining:    0.0s


[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.1s
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.0s
[CV] penalty=l2, C=0.0001 ............................................


...

[CV] ............. penalty=l2, C=0.0001, score=0.603834, total=   0.0s
[CV] penalty=l2, C=0.0001 ............................................
[CV] ............. penalty=l2, C=0.0001, score=0.577548, total=   0.0s
[CV] penalty=l1, C=0.000774263682681 .................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.6s remaining:    0.0s


[CV] .. penalty=l1, C=0.000774263682681, score=0.397680, total=   0.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.476869, total=   0.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.484104, total=   0.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.567789, total=   0.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.582487, total=   0.1s
[CV] penalty=l2, C=0.000774263682681 .................................
[CV] .. penalty=l2, C=0.000774263682681, score=0.539009, total=   0.0s
[CV] penalty=l2, C=0.000774263682681 .................................
[CV] .. penalty=l2, C=0.000774263682681, score=0.560357, total=   0.1s
[CV] penalty=l2, C=0.000774263682681 .................................
[CV] .

[CV] ...... penalty=l2, C=21.5443469003, score=0.691920, total=   0.1s
[CV] penalty=l2, C=21.5443469003 .....................................
[CV] ...... penalty=l2, C=21.5443469003, score=0.652767, total=   0.1s
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.767619, total=   0.6s
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.768457, total=   0.3s
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.715516, total=   0.5s
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.691318, total=   0.7s
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.652884, total=   0.5s
[CV] penalty=l2, C=166.81005372 ......................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   34.8s finished


# Mahanalobis

In [ ]:
result_file='result_grid_mh.csv'
dir_path='./'

In [ ]:
text_file = open(os.path.join(dir_path, result_file), 'w')

for k in range(2,15):
    
    for supportf in [0.95,0.99,0.995]:

        tuned_parameters = [{'penalty': ['l1','l2'],
                'C':list(np.logspace(-4, 5, 15))}]

        pipeline=Pipeline([('dim_red',PCA(n_components=k,svd_solver='arpack'))
                   ,('removeOutliers',OutlierMahalanobis(verbose=False,qqplot=False,support_fraction=supportf))])


        clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=1)
        fittedpipeline=pipeline.fit(np.array(dg_drop_minmax))

        data=fittedpipeline.transform(np.array(dg_drop_minmax))
        cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]

        clf.fit(data,cible_dropt)


        print('--------------------------', file=text_file)
        print("Components :"+str(k), file=text_file)
        print("Support Fraction :"+str(supportf), file=text_file)

        print('', file=text_file)
        print("Best parameters set found on development set:", file=text_file)
        print('', file=text_file)
        print(clf.best_params_, file=text_file)
        print('', file=text_file)

        print("Grid scores on development set:", file=text_file)
        print('', file=text_file)
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params), file=text_file)
        print('', file=text_file)

text_file.close()

# Sans rien

In [15]:
result_file='result_grid_without_2.csv'
dir_path='./'

In [16]:
text_file = open(os.path.join(dir_path, result_file), 'w')

tuned_parameters = [{'penalty': ['l1','l2'],
        'C':list(np.logspace(-4, 5, 15))}]


clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=1)

clf.fit(np.array(dg_drop_minmax),cible)


print('--------------------------', file=text_file)

print('', file=text_file)
print("Best parameters set found on development set:", file=text_file)
print('', file=text_file)
print(clf.best_params_, file=text_file)
print('', file=text_file)

print("Grid scores on development set:", file=text_file)
print('', file=text_file)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params), file=text_file)
print('', file=text_file)

text_file.close()

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 207.6min finished
